In [1]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
from plotly.graph_objs.scatter.marker import Line
from plotly.subplots import make_subplots

In [48]:
url = "https://drive.google.com/file/d/1KYq-55JHNJC9z5DljOnGzjFENGtU_whx/view?usp=sharing"

file_id = url.split('/')[-2]
dwn_url = "https://drive.google.com/uc?id=" + file_id

oftcdf = pd.read_csv(dwn_url, usecols=['Date','Leveraged Funds','Asset','Dealer'])

In [49]:
def oftcdf_reformat(d):
    d.columns = [x.lower().replace(' ','_') for x in d.columns]
    d['date'] = pd.to_datetime(d['date'])
    
    num_cols = ['leveraged_funds','asset','dealer']
    for c in num_cols:
        d[c] = d[c].str.replace(',','')
        d[c] = pd.to_numeric(d[c])
#         d[c] = d[c].astype(int).astype(float)
        
    return d

In [50]:
oftc = oftcdf_reformat(oftcdf)

In [51]:
oftc

,date,leveraged_funds,asset,dealer
0,2021-12-17,44689.0,584746.0,-449051.0
1,2021-12-10,16911.0,648940.0,-464295.0
2,2021-12-03,58122.0,681709.0,-478023.0
3,2021-11-26,-2533.0,770652.0,-496518.0
4,2021-11-19,-43262.0,774633.0,-483406.0
...,...,...,...,...
518,2012-01-13,-273829.0,432322.0,-229001.0
519,2012-01-06,-332155.0,427014.0,-213287.0
520,2011-12-30,-353090.0,458256.0,-184216.0
521,2011-12-23,-292618.0,383766.0,-159362.0


In [69]:
oftc = oftc.sort_values(by='date').reset_index()
oftc = oftc.assign(high14=lambda t: t.leveraged_funds.rolling(14).max(),
                  low14=lambda t: t.leveraged_funds.rolling(14).min(),
                  kperc=lambda t: (t.leveraged_funds - t.low14)/(t.high14 - t.low14),
                  davg=lambda t: t.kperc.rolling(3).mean())

In [58]:
oftc[:20]

,index,date,leveraged_funds,asset,dealer,high14,low14,kperc,davg
0,522,2011-12-16,-188987.0,394632.0,-111866.0,NaN,NaN,NaN,NaN
1,521,2011-12-23,-292618.0,383766.0,-159362.0,NaN,NaN,NaN,NaN
2,520,2011-12-30,-353090.0,458256.0,-184216.0,NaN,NaN,NaN,NaN
3,519,2012-01-06,-332155.0,427014.0,-213287.0,NaN,NaN,NaN,NaN
4,518,2012-01-13,-273829.0,432322.0,-229001.0,NaN,NaN,NaN,NaN
5,517,2012-01-20,-298909.0,499669.0,-262421.0,NaN,NaN,NaN,NaN
6,516,2012-01-27,-325276.0,573823.0,-340293.0,NaN,NaN,NaN,NaN
7,515,2012-02-03,-267026.0,573411.0,-316066.0,NaN,NaN,NaN,NaN
8,514,2012-02-10,-292811.0,663490.0,-342519.0,NaN,NaN,NaN,NaN
9,513,2012-02-17,-357546.0,730593.0,-370641.0,NaN,NaN,NaN,NaN


In [60]:
oftc.date.to_frame()

,date
0,2011-12-16
1,2011-12-23
2,2011-12-30
3,2012-01-06
4,2012-01-13
...,...
518,2021-11-19
519,2021-11-26
520,2021-12-03
521,2021-12-10


In [68]:
start = datetime(2012,4,27)

oftc.loc[max(oftc.index[oftc['date'] == start][0]-13,0)]['date']

Timestamp('2012-01-27 00:00:00')

In [70]:
comp_col_dict = {'spx':['E-Mini S&P 500 Futures','S&P 500 Price'],
                     'qqq':['E-Mini NASDAQ 100 Futures','QQQ Price'],
                     'dxy':['US Dollar Index Futures','US Dollar Index'],
                     'note10':['US 10y Treasury Note Futures','US 10y Yield'],
                     'wti':['Crude Oil WTI Futures','WTI Price']}

In [71]:
sel = 'E-Mini NASDAQ 100 Futures'

In [74]:
for k,v in comp_col_dict.items():
    if v[0] == sel:
        print(k)

qqq


In [76]:
[k for k,v in comp_col_dict.items() if v[0]==sel][0]

'qqq'